In [2]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , classification_report, r2_score
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [4]:
df = pd.read_csv("novagen_dataset.csv")
df.head(5)

,Age,BMI,Blood_Pressure,Cholesterol,Glucose_Level,Heart_Rate,Sleep_Hours,Exercise_Hours,Water_Intake,Stress_Level,...,Diet,MentalHealth,PhysicalActivity,MedicalHistory,Allergies,Diet_Type__Vegan,Diet_Type__Vegetarian,Blood_Group_AB,Blood_Group_B,Blood_Group_O
0,2.0,26.0,111.0,198.0,99.0,72.0,4.0,1.0,5.0,5.0,...,1,2,1,0,1,False,True,True,False,False
1,8.0,24.0,121.0,199.0,103.0,75.0,2.0,1.0,2.0,9.0,...,1,2,1,2,2,False,False,True,False,False
2,81.0,27.0,147.0,203.0,100.0,74.0,10.0,-0.0,5.0,1.0,...,2,0,0,1,0,True,False,False,False,False
3,25.0,21.0,150.0,199.0,102.0,70.0,7.0,3.0,3.0,3.0,...,1,2,1,2,0,True,False,False,True,False
4,24.0,26.0,146.0,202.0,99.0,76.0,10.0,2.0,5.0,1.0,...,2,0,2,0,2,False,True,False,True,False


In [25]:
from sklearn.preprocessing import LabelEncoder

le =LabelEncoder()

df["Diet_Type__Vegan"]=le.fit_transform(df["Diet_Type__Vegan"])
df["Diet_Type__Vegetarian"]=le.fit_transform(df["Diet_Type__Vegetarian"])
df.head()

,Age,BMI,Blood_Pressure,Cholesterol,Glucose_Level,Heart_Rate,Sleep_Hours,Exercise_Hours,Water_Intake,Stress_Level,...,Diet,MentalHealth,PhysicalActivity,MedicalHistory,Allergies,Diet_Type__Vegan,Diet_Type__Vegetarian,Blood_Group_AB,Blood_Group_B,Blood_Group_O
0,2.0,26.0,111.0,198.0,99.0,72.0,4.0,1.0,5.0,5.0,...,1,2,1,0,1,0,1,True,False,False
1,8.0,24.0,121.0,199.0,103.0,75.0,2.0,1.0,2.0,9.0,...,1,2,1,2,2,0,0,True,False,False
2,81.0,27.0,147.0,203.0,100.0,74.0,10.0,-0.0,5.0,1.0,...,2,0,0,1,0,1,0,False,False,False
3,25.0,21.0,150.0,199.0,102.0,70.0,7.0,3.0,3.0,3.0,...,1,2,1,2,0,1,0,False,True,False
4,24.0,26.0,146.0,202.0,99.0,76.0,10.0,2.0,5.0,1.0,...,2,0,2,0,2,0,1,False,True,False


In [6]:
numerical_cols = ["Blood_Pressure","Cholesterol","Stress_Level"]

from sklearn.impute import SimpleImputer

num_imp = SimpleImputer(strategy="mean")
df[numerical_cols] = num_imp.fit_transform(df[numerical_cols])

In [7]:
x = df.drop(columns=["Target","Blood_Group_AB","Blood_Group_B","Blood_Group_O"])
y = df["Target"]

In [36]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size =0.2, random_state =42
)


lr = LogisticRegression(max_iter=5000)
svc = SVC(max_iter=5000)
dt = DecisionTreeClassifier()

# Voting

In [37]:
vlf = VotingClassifier(
    estimators = [("lr",lr),("svc", svc), ("dt",dt)]
)

In [38]:
vlf.fit(x_train, y_train)

,estimators,"[('lr', ...), ('svc', ...), ...]"
,voting,'hard'
,weights,None
,n_jobs,None
,flatten_transform,True
,verbose,False
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True


In [39]:
y_pred = vlf.predict(x_test)
y_pred_train = clf_stack.predict(x_train)

print("accuracy :",accuracy_score(y_test, y_pred))
print("accuracy train:",accuracy_score(y_train, y_pred_train))
print(classification_report(y_test, y_pred))

accuracy : 0.8659685863874346
accuracy train: 0.885718025919623
              precision    recall  f1-score   support

           0       0.83      0.90      0.86       900
           1       0.90      0.84      0.87      1010

    accuracy                           0.87      1910
   macro avg       0.87      0.87      0.87      1910
weighted avg       0.87      0.87      0.87      1910



# Stacking

In [40]:
from sklearn.ensemble import StackingClassifier

In [41]:
from sklearn import preprocessing


In [42]:
meta_model = DecisionTreeClassifier()

clf_stack = StackingClassifier(
    estimators = [("lr",lr),("svc", svc), ("dt",dt)],
    final_estimator=meta_model, cv=5
)

In [43]:
clf_stack.fit(x_train, y_train)

,estimators,"[('lr', ...), ('svc', ...), ...]"
,final_estimator,DecisionTreeClassifier()
,cv,5
,stack_method,'auto'
,n_jobs,None
,passthrough,False
,verbose,0
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0


In [44]:
y_pred = clf_stack.predict(x_test)
y_pred_train = clf_stack.predict(x_train)

print("accuracy :",accuracy_score(y_test, y_pred))
print("accuracy train:",accuracy_score(y_train, y_pred_train))
print(classification_report(y_test, y_pred))

accuracy : 0.8434554973821989
accuracy train: 0.8999869092813195
              precision    recall  f1-score   support

           0       0.83      0.85      0.84       900
           1       0.86      0.84      0.85      1010

    accuracy                           0.84      1910
   macro avg       0.84      0.84      0.84      1910
weighted avg       0.84      0.84      0.84      1910



# XGBoost

In [45]:
import xgboost as xgb

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

xgb_clf = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    eval_metric="logloss",
    random_state=42
)

xgb_clf.fit(x_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [46]:
y_pred = xgb_clf.predict(x_test)
y_pred_train = clf_stack.predict(x_train)

print("accuracy :",accuracy_score(y_test, y_pred))
print("accuracy train:",accuracy_score(y_train, y_pred_train))

accuracy : 0.9193717277486911
accuracy train: 0.8999869092813195


In [47]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91       900
           1       0.92      0.93      0.92      1010

    accuracy                           0.92      1910
   macro avg       0.92      0.92      0.92      1910
weighted avg       0.92      0.92      0.92      1910

